In [1]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          48 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                AuthenticAMD
  Model name:             AMD EPYC 7B12
    CPU family:           23
    Model:                49
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4499.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall
                           nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep
                          _good nopl nonstop_tsc cpuid extd_apicid tsc_known_fre
                          q pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic mo
                          vbe popcnt aes xsave avx f16c rdrand hypervisor lahf_l

**----------Load The Data---------**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math
import random
import numpy as np
from numpy import ndarray

In [4]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 7 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [5]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/higgs_ds_mod_orig.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [6]:
# Data sizes:
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
LQ = 500       # Large Query: 500 queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (LB, SQ)
]

In [20]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


**--------QuickSelect Algorithm----------------**

In [8]:
from collections import deque
from typing import List, Tuple

#*-----------------------------------------------------------------------------
def quickselect(distances:List, k:int):
    """QuickSelect algorithm to find the k-th smallest distance."""

    if len(distances) == 1:
        return distances[0]

    if len(distances) == 0:
        return None

    stack = deque()
    stack.append( (distances, k) )
    while stack:

        distances, k = stack.pop()
        if len(distances) == 1:
            return distances[0]

        # Returned as a tuple
        pivot = median_of_medians(distances)
        lows, highs, pivots = partition(distances, pivot)

        if len(lows) > k:
            stack.append((lows, k))
        elif len(lows) + len(pivots) > k:
            return pivot
        else:
            stack.append((highs, k - len(lows) - len(pivots)))
    return pivot

#*-----------------------------------------------------------------------------
def partition(distances, pivot):
    lows, highs, pivots = [], [], []

    for d in distances:
        if d[0] < pivot[0]:
            lows.append(d)
        elif d[0] > pivot[0]:
            highs.append(d)
        else:
            pivots.append(d)
    return lows, highs, pivots

#*-----------------------------------------------------------------------------
def median_of_medians(distances, grp_size=5):
    """Find the index of the median distance in a list of distances.
    Returns the index and the median distance as a tuple."""

    stack = deque()
    # Split the list in chunks of grp_size
    # And add the lists to the stack on the right side
    for i in range(0, len(distances), grp_size):
        subList = distances[i:i+grp_size]
        insertion_sort(subList)
        stack.append(subList)

    # We want to find the median of the medians
    # and return the index of that median
    median_list = []
    stack_count = len(stack)
    while 0 < stack_count:

        subList = stack.pop()
        stack_count -= 1

        mid_index = len(subList) // 2
        median_list.append(subList[mid_index])

        if len(median_list) == grp_size:
            insertion_sort(median_list)
            # append to the left, since we are popping from the right
            stack.appendleft(median_list)
            if stack_count == 0:
                stack_count = len(stack)
            median_list = []
        # If the count is 0, we have reached the end of the old stack
        # and need to insert the remaining elements,
        # stack count and the median list will be reset
        if stack_count == 0 and len(median_list) > 0:
            if len(median_list) > 1:
                insertion_sort(median_list)
            stack.appendleft(median_list)
            stack_count = len(stack)
            if stack_count == 1 and len(median_list) == 1:
                return median_list[0]
            median_list = []

#*-----------------------------------------------------------------------------
def insertion_sort(arr):
# In-place insertion sort, efficient for small arrays
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j][0] > key[0]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
#*-----------------------------------------------------------------------------

**----------VP Tree---------------------**

In [9]:
import numpy as np
from collections import deque
#*-----------------------------------------------------------------------------

class VPNode:
    def __init__(self, point=None):
        self.point = point
        self.mu = 0
        self.threshold_points = []
        self.left_tr = None
        self.right_tr = None
        self.points = []
        self.centroid = None
        self.radius = 0

        # Cache for numpy arrays
        self._np_points = None
        self._np_threshold_points = None
#*-----------------------------------------------------------------------------

class VPTree:
    def __init__(self, leaf_size=50, epsilon=0.1):
        self.root = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def build(self, points):
        self.root = VPNode()
        stack = deque([(self.root, points)])

        while stack:
            node, pts = stack.pop()
            n_pts = len(pts)

            if n_pts <= self.leaf_size:
                node.points = [p for p in pts]
                node.centroid = np.mean(pts, axis=0)
                node.radius = np.max(np.linalg.norm(pts - node.centroid, axis=1))
                continue

            node.point = pts[0]
            remaining_pts = pts[1:]

            if len(remaining_pts) == 0:
                node.points = [node.point]
                node.centroid = node.point
                node.radius = 0
                continue

            distances = np.linalg.norm(remaining_pts - node.point, axis=1)
            median_idx = len(distances) // 2
            dist_tuples = [(float(distances[i]), i) for i in range(len(distances))]
            median_tuple = quickselect(dist_tuples, median_idx)
            node.mu = median_tuple[0]

            threshold_mask = (distances > node.mu - self.epsilon) & (distances < node.mu + self.epsilon)
            threshold_pts = remaining_pts[threshold_mask]

            if len(threshold_pts) > 0 and len(threshold_pts) < self.leaf_size:
                node.threshold_points = [p for p in threshold_pts]
                node.centroid = np.mean(threshold_pts, axis=0)
                node.radius = np.max(np.linalg.norm(threshold_pts - node.centroid, axis=1))

            inner_mask = distances <= node.mu - self.epsilon
            outer_mask = distances >= node.mu + self.epsilon
            inner_pts = remaining_pts[inner_mask]
            outer_pts = remaining_pts[outer_mask]

            if len(inner_pts) > 0:
                node.left_tr = VPNode()
                stack.appendleft((node.left_tr, inner_pts))

            if len(outer_pts) > 0:
                node.right_tr = VPNode()
                stack.appendleft((node.right_tr, outer_pts))

        return self.root

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Convert threshold points to numpy array
            if node.threshold_points:
                node._np_threshold_points = np.array(node.threshold_points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Performs an efficient nearest neighbor search using pre-converted numpy arrays.
        """
        # Ensure tree is prepared for search

        best_dist = float("inf")
        best_point = None
        stack = [self.root]

        while stack:
            node = stack.pop()
            if node is None:
                continue

            # Check leaf node points
            if node.points:
                if node.centroid is not None:
                    dist_to_centroid = np.linalg.norm(query - node.centroid)
                    if dist_to_centroid - node.radius >= best_dist:
                        continue

                # Use cached numpy array for fast vector operations
                dists = np.linalg.norm(node._np_points - query, axis=1)
                min_idx = np.argmin(dists)
                if dists[min_idx] < best_dist:
                    best_dist = dists[min_idx]
                    best_point = node._np_points[min_idx]
                continue

            # Check vantage point
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.point

            # Check threshold points
            if node.threshold_points:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius < best_dist:
                    # Use cached numpy array for threshold points
                    dists = np.linalg.norm(node._np_threshold_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_threshold_points[min_idx]

            # Determine which subtree to explore
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                dist_to_threshold = abs(dist_to_vp - node.mu)

                if dist_to_vp <= node.mu - self.epsilon:
                    # Query is in inner region
                    if node.left_tr:
                        stack.append(node.left_tr)
                    if node.right_tr and dist_to_threshold < best_dist:
                        stack.append(node.right_tr)
                elif dist_to_vp >= node.mu + self.epsilon:
                    # Query is in outer region
                    if node.right_tr:
                        stack.append(node.right_tr)
                    if node.left_tr and dist_to_threshold < best_dist:
                        stack.append(node.left_tr)
                else:
                    # Query is in threshold band
                    if dist_to_vp <= node.mu:
                        # Prioritize inner subtree
                        if node.left_tr:
                            stack.append(node.left_tr)
                        if node.right_tr:
                            stack.append(node.right_tr)
                    else:
                        # Prioritize outer subtree
                        if node.right_tr:
                            stack.append(node.right_tr)
                        if node.left_tr:
                            stack.append(node.left_tr)

        return best_dist, best_point

    #*-----------------------------------------------------------------------------
    def insert(self, point):
        """
        Insert a new point into the VP-Tree with fast list-based operations.
        """

        if self.root is None:
            self.root = VPNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            return

        current = self.root

        while not current.points:
            if current.point is None:
                if current.left_tr:
                    current = current.left_tr
                elif current.right_tr:
                    current = current.right_tr
                else:
                    current.points = [point]
                    current.centroid = point
                    current.radius = 0
                    return
            else:
                dist_to_vp = np.linalg.norm(point - current.point)

                if abs(dist_to_vp - current.mu) <= self.epsilon:
                    if not current.threshold_points:
                        current.threshold_points = [point]
                        current.centroid = point
                        current.radius = 0
                        return
                    elif len(current.threshold_points) < self.leaf_size:
                        n = len(current.threshold_points)
                        current.threshold_points.append(point)
                        current.centroid = (n * current.centroid + point) / (n + 1)
                        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
                        return

                if dist_to_vp < current.mu - self.epsilon:
                    if current.left_tr:
                        current = current.left_tr
                    else:
                        current.left_tr = VPNode()
                        current.left_tr.points = [point]
                        current.left_tr.centroid = point
                        current.left_tr.radius = 0
                        return
                else:
                    if current.right_tr:
                        current = current.right_tr
                    else:
                        current.right_tr = VPNode()
                        current.right_tr.points = [point]
                        current.right_tr.centroid = point
                        current.right_tr.radius = 0
                        return

        n = len(current.points)
        # Dynamically adjust the node size for homogeneous point storage
        if n == 2*self.leaf_size:
            self.leaf_size = n

        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
#*------------------------------------------------------------------------------------------

**------VP Tree Node Size Optimization For Rebuild------**

----------------------

**------------------VP Dyn Wider Scope Run 1--------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

# Define leaf size range
leaf_size_range_start = 100
leaf_size_range_end = 10_000  # Ensure inclusive range
leaf_size_step = 100

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 30.79it/s]


-------------------------------------------------------------
Build Time: 23.7661 sec, Search Time: 1.0140 sec
Leaf Size: 100, Total Time: 24.7802 sec

2: === Testing with Leaf Size: 200, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 43.66it/s]


-------------------------------------------------------------
Build Time: 21.4736 sec, Search Time: 0.7451 sec
Leaf Size: 200, Total Time: 22.2187 sec

3: === Testing with Leaf Size: 300, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 13.3166 sec, Search Time: 0.6704 sec
Leaf Size: 9,800, Total Time: 13.9869 sec

99: === Testing with Leaf Size: 9,900, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 48.06it/s]


-------------------------------------------------------------
Build Time: 13.2907 sec, Search Time: 0.6668 sec
Leaf Size: 9,900, Total Time: 13.9575 sec

100: === Testing with Leaf Size: 10,000, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 47.81it/s]

-------------------------------------------------------------
Build Time: 13.0439 sec, Search Time: 0.6647 sec
Leaf Size: 10,000, Total Time: 13.7086 sec

=== Summary of VP Tree Node Size Test ===
Leaf Size: 100, Build Time: 23.7661 sec, Search Time: 1.0140 sec, Total Time: 24.7802 sec
Leaf Size: 200, Build Time: 21.4736 sec, Search Time: 0.7451 sec, Total Time: 22.2187 sec
Leaf Size: 300, Build Time: 20.0326 sec, Search Time: 0.6381 sec, Total Time: 20.6707 sec
Leaf Size: 400, Build Time: 19.8898 sec, Search Time: 0.6233 sec, Total Time: 20.5131 sec
Leaf Size: 500, Build Time: 18.4147 sec, Search Time: 0.5577 sec, Total Time: 18.9724 sec
Leaf Size: 600, Build Time: 18.1581 sec, Search Time: 0.5581 sec, Total Time: 18.7162 sec
Leaf Size: 700, Build Time: 18.3884 sec, Search Time: 0.5575 sec, Total Time: 18.9459 sec
Leaf Size: 800, Build Time: 18.4012 sec, Search Time: 0.5608 sec, Total Time: 18.9621 sec
Leaf Size: 900, Build Time: 18.3306 sec, Search Time: 0.5797 sec, Total Time: 18.91

**---------Wider scope run 2-----------**

In [ ]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

# Define leaf size range
leaf_size_range_start = 100
leaf_size_range_end = 10_000  # Ensure inclusive range
leaf_size_step = 100

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 27.84it/s]


-------------------------------------------------------------
Build Time: 23.7511 sec, Search Time: 1.0395 sec
Leaf Size: 100, Total Time: 24.7906 sec

2: === Testing with Leaf Size: 200, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 39.60it/s]


-------------------------------------------------------------
Build Time: 21.6699 sec, Search Time: 0.7792 sec
Leaf Size: 200, Total Time: 22.4491 sec

3: === Testing with Leaf Size: 300, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 13.4411 sec, Search Time: 0.7043 sec
Leaf Size: 9,800, Total Time: 14.1454 sec

99: === Testing with Leaf Size: 9,900, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 42.64it/s]


-------------------------------------------------------------
Build Time: 13.3536 sec, Search Time: 0.7201 sec
Leaf Size: 9,900, Total Time: 14.0737 sec

100: === Testing with Leaf Size: 10,000, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 45.29it/s]

-------------------------------------------------------------
Build Time: 13.2656 sec, Search Time: 0.6922 sec
Leaf Size: 10,000, Total Time: 13.9577 sec

=== Summary of VP Tree Node Size Test ===
Leaf Size: 100, Build Time: 23.7511 sec, Search Time: 1.0395 sec, Total Time: 24.7906 sec
Leaf Size: 200, Build Time: 21.6699 sec, Search Time: 0.7792 sec, Total Time: 22.4491 sec
Leaf Size: 300, Build Time: 20.0073 sec, Search Time: 0.6444 sec, Total Time: 20.6517 sec
Leaf Size: 400, Build Time: 20.0159 sec, Search Time: 0.6776 sec, Total Time: 20.6935 sec
Leaf Size: 500, Build Time: 18.5238 sec, Search Time: 0.5827 sec, Total Time: 19.1064 sec
Leaf Size: 600, Build Time: 18.4087 sec, Search Time: 0.5774 sec, Total Time: 18.9861 sec
Leaf Size: 700, Build Time: 18.6316 sec, Search Time: 0.5820 sec, Total Time: 19.2136 sec
Leaf Size: 800, Build Time: 18.6339 sec, Search Time: 0.5816 sec, Total Time: 19.2156 sec
Leaf Size: 900, Build Time: 18.6806 sec, Search Time: 0.5858 sec, Total Time: 19.26

**----------Wider Scope Opt Node Size-----------**

In [ ]:
import numpy as np

# Leaf sizes
leaf_sizes = np.array([  100,  200,  300,  400,  500,  600,  700,  800,  900, 1000, 1100, 1200,
  1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400,
  2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600,
  3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800,
  4900, 5000, 5100, 5200, 5300, 5400, 5500, 5600, 5700, 5800, 5900, 6000,
  6100, 6200, 6300, 6400, 6500, 6600, 6700, 6800, 6900, 7000, 7100, 7200,
  7300, 7400, 7500, 7600, 7700, 7800, 7900, 8000, 8100, 8200, 8300, 8400,
  8500, 8600, 8700, 8800, 8900, 9000, 9100, 9200, 9300, 9400, 9500, 9600,
  9700, 9800, 9900,10000])

# Total times from two runs (or versions)
total_times_1 = np.array([24.7802,22.2187,20.6707,20.5131,18.9724,18.7162,18.9459,18.9621,18.9102,
 17.6156,17.6328,17.7667,17.5227,17.5565,17.6622,17.3265,17.5348,17.3434,
 17.479 ,16.3388,16.3645,16.3618,16.3179,16.3426,16.2555,16.3802,16.3688,
 16.5088,16.6688,16.6787,16.6242,16.6448,16.5091,16.6268,16.5278,16.8081,
 16.7759,16.7013,16.7775,15.1723,15.3794,15.3544,15.4396,15.4615,15.4341,
 15.3057,15.3588,15.3514,15.4919,15.4543,15.2522,15.2777,15.3537,15.2872,
 15.3431,15.412 ,15.1481,15.0725,14.9909,15.1761,14.9966,15.0085,15.0945,
 15.1764,15.0632,15.1886,14.9616,14.9441,14.9542,15.0559,15.0484,15.1366,
 15.106 ,15.2489,15.1395,15.0407,14.9719,14.8726,13.7502,13.8989,13.7938,
 13.8678,13.7802,13.7649,13.6827,13.7297,13.7555,13.8479,13.9173,13.633 ,
 13.7459,13.872 ,13.8514,13.7457,13.6308,13.7192,13.8274,13.9869,13.9575,
 13.7086])

total_times_2 = np.array([24.7906,22.4491,20.6517,20.6935,19.1064,18.9861,19.2136,19.2156,19.2665,
 17.6521,17.8069,17.8196,17.8502,17.7357,17.7668,17.899 ,17.8356,17.9523,
 17.8228,16.6982,16.3723,16.648 ,16.6199,16.7429,16.7198,16.6507,16.6501,
 16.6146,16.686 ,16.6922,16.7756,16.5437,16.7814,16.8194,16.7288,16.5579,
 16.5546,16.7071,16.8178,15.4073,15.5401,15.4307,15.5078,15.4859,15.4002,
 15.5903,15.2935,15.356 ,15.3469,15.2111,15.2842,15.4752,15.3783,15.4034,
 15.4304,15.1561,15.2163,15.1833,15.1293,15.1451,15.2039,15.1209,15.4294,
 15.1906,15.1793,15.1818,15.3273,15.1423,15.1472,15.436 ,15.2714,15.2461,
 15.3226,15.1005,15.1735,15.1446,15.2584,15.1498,13.8942,13.9633,14.2845,
 14.1372,14.092 ,14.0972,14.0558,14.068 ,13.928 ,14.0571,13.9158,14.1449,
 13.9713,13.8068,13.8574,14.0268,13.7799,14.0296,13.9908,14.1454,14.0737,
 13.9577])

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_leaf_size = leaf_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at leaf size: {optimal_leaf_size}")


Average total time (sec): 13.7053
Minimum found at leaf size: 9500


---------------------------------------

**-----------Epsilon Optimization For Rebuild---------**

**--------Wider Scope Run 1--------------**

In [15]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size = 9500
# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 30
epsilon_size_step = 0.5

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 66.02it/s]


-------------------------------------------------------------
Build Time: 9.4934 sec, Search Time: 0.4884 sec
Epsilon: 0.0, Total Time: 9.9818 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:00<00:00, 328.96it/s]


-------------------------------------------------------------
Build Time: 2.6721 sec, Search Time: 0.0769 sec
Epsilon: 0.5, Total Time: 2.7491 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 1.4630 sec, Search Time: 0.0020 sec
Epsilon: 29.0, Total Time: 1.4651 sec

60: === Testing with Epsilon: 29.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 29.5): 100%|██████████| 20/20 [00:00<00:00, 59578.18it/s]


-------------------------------------------------------------
Build Time: 1.4713 sec, Search Time: 0.0026 sec
Epsilon: 29.5, Total Time: 1.4739 sec

61: === Testing with Epsilon: 30.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 30.0): 100%|██████████| 20/20 [00:00<00:00, 20810.24it/s]

-------------------------------------------------------------
Build Time: 1.4987 sec, Search Time: 0.0037 sec
Epsilon: 30.0, Total Time: 1.5024 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 9.4934 sec, Search Time: 0.4884 sec, Total Time: 9.9818 sec
Epsilon: 0.5, Build Time: 2.6721 sec, Search Time: 0.0769 sec, Total Time: 2.7491 sec
Epsilon: 1.0, Build Time: 1.7927 sec, Search Time: 0.0176 sec, Total Time: 1.8103 sec
Epsilon: 1.5, Build Time: 1.5724 sec, Search Time: 0.0115 sec, Total Time: 1.5839 sec
Epsilon: 2.0, Build Time: 1.4652 sec, Search Time: 0.0100 sec, Total Time: 1.4752 sec
Epsilon: 2.5, Build Time: 1.4756 sec, Search Time: 0.0052 sec, Total Time: 1.4809 sec
Epsilon: 3.0, Build Time: 1.5073 sec, Search Time: 0.0037 sec, Total Time: 1.5110 sec
Epsilon: 3.5, Build Time: 1.5266 sec, Search Time: 0.0040 sec, Total Time: 1.5307 sec
Epsilon: 4.0, Build Time: 1.5160 sec, Search Time: 0.0034 sec, Total Time: 1.5194 sec
Epsilon: 4.5, Build Time: 1.5056

**--------Wider Scope Run 2-------**

In [17]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size = 9500
# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 30
epsilon_size_step = 0.5

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 64.05it/s]


-------------------------------------------------------------
Build Time: 9.5556 sec, Search Time: 0.4815 sec
Epsilon: 0.0, Total Time: 10.0371 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:00<00:00, 331.78it/s]


-------------------------------------------------------------
Build Time: 2.6920 sec, Search Time: 0.0761 sec
Epsilon: 0.5, Total Time: 2.7681 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 1.4836 sec, Search Time: 0.0026 sec
Epsilon: 29.0, Total Time: 1.4863 sec

60: === Testing with Epsilon: 29.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 29.5): 100%|██████████| 20/20 [00:00<00:00, 36472.21it/s]


-------------------------------------------------------------
Build Time: 1.5149 sec, Search Time: 0.0023 sec
Epsilon: 29.5, Total Time: 1.5172 sec

61: === Testing with Epsilon: 30.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 30.0): 100%|██████████| 20/20 [00:00<00:00, 37249.59it/s]

-------------------------------------------------------------
Build Time: 1.4994 sec, Search Time: 0.0022 sec
Epsilon: 30.0, Total Time: 1.5017 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 9.5556 sec, Search Time: 0.4815 sec, Total Time: 10.0371 sec
Epsilon: 0.5, Build Time: 2.6920 sec, Search Time: 0.0761 sec, Total Time: 2.7681 sec
Epsilon: 1.0, Build Time: 1.7162 sec, Search Time: 0.0171 sec, Total Time: 1.7333 sec
Epsilon: 1.5, Build Time: 1.5439 sec, Search Time: 0.0103 sec, Total Time: 1.5542 sec
Epsilon: 2.0, Build Time: 1.5025 sec, Search Time: 0.0100 sec, Total Time: 1.5124 sec
Epsilon: 2.5, Build Time: 1.5033 sec, Search Time: 0.0041 sec, Total Time: 1.5074 sec
Epsilon: 3.0, Build Time: 1.5062 sec, Search Time: 0.0036 sec, Total Time: 1.5098 sec
Epsilon: 3.5, Build Time: 1.5048 sec, Search Time: 0.0035 sec, Total Time: 1.5082 sec
Epsilon: 4.0, Build Time: 1.5248 sec, Search Time: 0.0027 sec, Total Time: 1.5275 sec
Epsilon: 4.5, Build Time: 1.499

**---------Opt Epsilon For Rebuild Wider Scope--------**

In [18]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array(
    [ 0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5,
  7. , 7.5, 8. , 8.5, 9. , 9.5,10. ,10.5,11. ,11.5,12. ,12.5,13. ,13.5,
 14. ,14.5,15. ,15.5,16. ,16.5,17. ,17.5,18. ,18.5,19. ,19.5,20. ,20.5,
 21. ,21.5,22. ,22.5,23. ,23.5,24. ,24.5,25. ,25.5,26. ,26.5,27. ,27.5,
 28. ,28.5,29. ,29.5,30. ]
    )

# Total times from two runs (or versions)
total_times_1 = np.array(
    [10.0371, 2.7681, 1.7333, 1.5542, 1.5124, 1.5074, 1.5098, 1.5082, 1.5275,
  1.502 , 1.4827, 1.5289, 1.4994, 1.5017, 1.4979, 1.525 , 1.5358, 1.5212,
  1.5164, 1.567 , 1.4908, 1.5017, 1.4877, 1.4982, 1.5127, 1.5075, 1.516 ,
  1.5429, 1.504 , 1.4705, 1.5644, 1.5783, 1.5122, 1.5292, 1.5342, 1.5283,
  1.5173, 1.5042, 1.5094, 1.5316, 1.496 , 1.5112, 1.5609, 1.5235, 1.5344,
  1.5126, 1.5381, 1.5055, 1.5212, 1.5765, 1.512 , 1.4914, 1.5056, 1.534 ,
  1.5724, 1.5098, 1.5542, 1.5767, 1.4863, 1.5172, 1.5017]
                         )

total_times_2 = np.array(
    [9.9818,2.7491,1.8103,1.5839,1.4752,1.4809,1.511 ,1.5307,1.5194,1.5091,
 1.492 ,1.5102,1.4837,1.4897,1.5017,1.5134,1.5104,1.5196,1.5125,1.5704,
 1.4813,1.5419,1.4993,1.5363,1.5135,1.5002,1.5007,1.493 ,1.4696,1.5246,
 1.4878,1.4731,1.4477,1.4863,1.49  ,1.4612,1.4984,1.4811,1.5051,1.5021,
 1.5013,1.5597,1.5181,1.486 ,1.4949,1.4794,1.4736,1.4714,1.5154,1.5588,
 1.4862,1.5032,1.479 ,1.5244,1.5073,1.4778,1.503 ,1.4717,1.4651,1.4739,
 1.5024]
    )

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 1.4757
Minimum found at epsilon size: 29.0


**------Epsilon Optimization For Insertion------------**

**----------Epsilon For Insert Wider Scope Run 1-----------**

In [19]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size_rebuild = 9500
epsilon_rebuild = 29.0

# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 30
epsilon_size_step = 0.5

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity
# Readjust for the insertion
data_size, query_size = test_combinations[0]

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    vptree.epsilon = epsilon

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting VP Tree"):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 24.81it/s]


-------------------------------------------------------------
Build Time: 8.3195 sec, Search Time: 0.9874 sec
Epsilon: 0.0, Total Time: 9.3070 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:00<00:00, 32.75it/s]


-------------------------------------------------------------
Build Time: 8.3606 sec, Search Time: 0.8945 sec
Epsilon: 0.5, Total Time: 9.2551 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 8.3772 sec, Search Time: 1.0103 sec
Epsilon: 29.0, Total Time: 9.3875 sec

60: === Testing with Epsilon: 29.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 29.5): 100%|██████████| 20/20 [00:00<00:00, 24.41it/s]


-------------------------------------------------------------
Build Time: 8.4237 sec, Search Time: 0.9867 sec
Epsilon: 29.5, Total Time: 9.4104 sec

61: === Testing with Epsilon: 30.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 30.0): 100%|██████████| 20/20 [00:00<00:00, 23.97it/s]

-------------------------------------------------------------
Build Time: 8.4051 sec, Search Time: 1.0100 sec
Epsilon: 30.0, Total Time: 9.4151 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 8.3195 sec, Search Time: 0.9874 sec, Total Time: 9.3070 sec
Epsilon: 0.5, Build Time: 8.3606 sec, Search Time: 0.8945 sec, Total Time: 9.2551 sec
Epsilon: 1.0, Build Time: 8.4519 sec, Search Time: 0.9305 sec, Total Time: 9.3824 sec
Epsilon: 1.5, Build Time: 8.4282 sec, Search Time: 1.0365 sec, Total Time: 9.4647 sec
Epsilon: 2.0, Build Time: 8.4581 sec, Search Time: 1.0141 sec, Total Time: 9.4722 sec
Epsilon: 2.5, Build Time: 8.4397 sec, Search Time: 1.0396 sec, Total Time: 9.4793 sec
Epsilon: 3.0, Build Time: 8.3501 sec, Search Time: 1.0492 sec, Total Time: 9.3992 sec
Epsilon: 3.5, Build Time: 8.3276 sec, Search Time: 1.0510 sec, Total Time: 9.3786 sec
Epsilon: 4.0, Build Time: 8.4806 sec, Search Time: 1.0585 sec, Total Time: 9.5391 sec
Epsilon: 4.5, Build Time: 8.4898

**----------Epsilon For Insert Wider Scope Run 2---------**

In [21]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size_rebuild = 9500
epsilon_rebuild = 29.0

# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 30
epsilon_size_step = 0.5

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity
# Readjust for the insertion
data_size, query_size = test_combinations[0]

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    vptree.epsilon = epsilon

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting VP Tree"):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 24.50it/s]


-------------------------------------------------------------
Build Time: 8.4091 sec, Search Time: 1.0118 sec
Epsilon: 0.0, Total Time: 9.4209 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:00<00:00, 29.74it/s]


-------------------------------------------------------------
Build Time: 8.4239 sec, Search Time: 0.9700 sec
Epsilon: 0.5, Total Time: 9.3938 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 8.3505 sec, Search Time: 1.0143 sec
Epsilon: 29.0, Total Time: 9.3648 sec

60: === Testing with Epsilon: 29.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 29.5): 100%|██████████| 20/20 [00:00<00:00, 24.09it/s]


-------------------------------------------------------------
Build Time: 8.3689 sec, Search Time: 1.0093 sec
Epsilon: 29.5, Total Time: 9.3781 sec

61: === Testing with Epsilon: 30.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 30.0): 100%|██████████| 20/20 [00:00<00:00, 24.40it/s]

-------------------------------------------------------------
Build Time: 8.4014 sec, Search Time: 0.9961 sec
Epsilon: 30.0, Total Time: 9.3974 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 8.4091 sec, Search Time: 1.0118 sec, Total Time: 9.4209 sec
Epsilon: 0.5, Build Time: 8.4239 sec, Search Time: 0.9700 sec, Total Time: 9.3938 sec
Epsilon: 1.0, Build Time: 8.3676 sec, Search Time: 1.0233 sec, Total Time: 9.3909 sec
Epsilon: 1.5, Build Time: 8.4234 sec, Search Time: 1.0246 sec, Total Time: 9.4480 sec
Epsilon: 2.0, Build Time: 8.4220 sec, Search Time: 1.0459 sec, Total Time: 9.4679 sec
Epsilon: 2.5, Build Time: 8.4109 sec, Search Time: 1.0457 sec, Total Time: 9.4566 sec
Epsilon: 3.0, Build Time: 8.3946 sec, Search Time: 1.0387 sec, Total Time: 9.4333 sec
Epsilon: 3.5, Build Time: 8.3264 sec, Search Time: 1.0572 sec, Total Time: 9.3837 sec
Epsilon: 4.0, Build Time: 8.4878 sec, Search Time: 1.0468 sec, Total Time: 9.5346 sec
Epsilon: 4.5, Build Time: 8.3869

**---------Opt Epsilon For Insertion Wide Scope------------**

In [22]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array(
    [ 0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5,
  7. , 7.5, 8. , 8.5, 9. , 9.5,10. ,10.5,11. ,11.5,12. ,12.5,13. ,13.5,
 14. ,14.5,15. ,15.5,16. ,16.5,17. ,17.5,18. ,18.5,19. ,19.5,20. ,20.5,
 21. ,21.5,22. ,22.5,23. ,23.5,24. ,24.5,25. ,25.5,26. ,26.5,27. ,27.5,
 28. ,28.5,29. ,29.5,30. ]
    )

# Total times from two runs (or versions)
total_times_1 = np.array(
    [9.4209,9.3938,9.3909,9.448 ,9.4679,9.4566,9.4333,9.3837,9.5346,9.4349,
 9.4061,9.431 ,9.4513,9.4718,9.4945,9.3532,9.4669,9.4454,9.5955,9.4729,
 9.4111,9.384 ,9.3698,9.3828,9.4742,9.3811,9.4591,9.3894,9.2895,9.3364,
 9.438 ,9.3413,9.3722,9.4228,9.5634,9.3464,9.4782,9.4192,9.3521,9.3936,
 9.3589,9.3246,9.4069,9.4601,9.6029,9.4478,9.3351,9.3582,9.347 ,9.4236,
 9.6117,9.3996,9.4449,9.3972,9.3208,9.5648,9.356 ,9.3935,9.3648,9.3781,
 9.3974]
    )

total_times_2 = np.array(
    [9.307 ,9.2551,9.3824,9.4647,9.4722,9.4793,9.3992,9.3786,9.5391,9.5338,
 9.3923,9.3979,9.6229,9.3591,9.4793,9.4153,9.3874,9.433 ,9.3704,9.4895,
 9.501 ,9.4452,9.5139,9.4661,9.511 ,9.4656,9.5786,9.431 ,9.4892,9.545 ,
 9.4405,9.4103,9.4438,9.3613,9.5291,9.3315,9.5786,9.3962,9.5377,9.4609,
 9.4083,9.455 ,9.3822,9.4404,9.3218,9.4533,9.4662,9.3993,9.4144,9.5218,
 9.4849,9.4006,9.4933,9.692 ,9.3948,9.3751,9.4203,9.4548,9.3875,9.4104,
 9.4151]
    )

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 9.3245
Minimum found at epsilon size: 0.5
